In [1]:
'''
I have used the movies dataset(100K) since the song dataset(1000K) which I used in method 1,method 2 was quite large
This method uses the keras library with neural network
We will try to make features of user and item using known rating
We will then merge the features of user and item and fed it to neural network
We will get the output of desired rating from neural network
We will then use these learnt features to predict new rating
'''
#Importing necessary liraries
import pandas as pd
import keras
from keras.optimizers import Adam
from keras.constraints import non_neg
import numpy as np
import keras.models as kmodels
import keras.layers as klayers
import keras.backend as K
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.layers import concatenate


Using TensorFlow backend.


In [2]:

#Collecting the necessary movielens data
data=pd.read_csv(r"C:\Users\shashank\Desktop\music recommendation\ml-100k\u.data" \
,sep='\t',names="user_id,item_id,rating,timestamp".split(","))

'''
data.user_id = data.user_id.astype('category').cat.codes.values
data.item_id = data.item_id.astype('category').cat.codes.values
'''
print('###Size of our full dataset###')
print()
print(data.shape)

###Size of our full dataset###

(100000, 4)


In [3]:
#all_user contains id of all users
all_user = data['user_id'].unique().tolist()
#all_item contains id of all items
all_item = data['item_id'].unique().tolist()

#dict1(dictionary) contains modified ids of users between 0 and len(all_user)
dict1 = {j:i for i,j in enumerate(all_user)} 
#dict2(dictionary) contains modified ids of users between 0 and len(all_item)
dict2 = {j:i for i,j in enumerate(all_item)} 

#below for loop will convert old ids to new ids present in dict1 and dict2
for item,row in data.iterrows():
	row['user_id'] = dict1[row['user_id']]
	row['item_id'] = dict2[row['item_id']]

#Splitting of data
train, test = train_test_split(data, test_size=0.2)
print()
print('###Size of our training dataset')
print(train.shape)
print()
print('###Size of our test dataset')
print(test.shape)
print()


###Size of our training dataset
(80000, 4)

###Size of our test dataset
(20000, 4)



In [10]:
#users contain all unique user id
users = len(data.user_id.unique())

#movies contain all unique movie id
movies = len(data.item_id.unique()) 
print('')

#total features we would like to learn for user and movie
features = 30

movie = keras.layers.Input(shape=[1],name='movie_input')

#this creates a feature space for item

feature_space_movie = keras.layers.Embedding(movies , features, name='movie_feature_space',embeddings_constraint=non_neg())(movie)
movie_flattened = keras.layers.Flatten()(feature_space_movie)

user = keras.layers.Input(shape=[1],name='user_input')

#this creates feature space for user
feature_space_user = keras.layers.Embedding(users, features,name='User_Feature_space',embeddings_constraint=non_neg())(user)
user_flattened = keras.layers.Flatten()(feature_space_user)

#lets merge features of user and movie to fed to neural network

model = concatenate([movie_flattened,user_flattened],axis=-1)

#fedding to neural network

model = keras.layers.Dense(45,activation='relu')(model)

model = keras.layers.Dropout(0.35)(model)

model = keras.layers.Dense(25,activation='relu')(model)

model = keras.layers.Dropout(0.25)(model)

model = keras.layers.Dense(1)(model)

result = keras.Model(inputs=[user, movie],outputs=[model])

result.compile('adam', 'mse')


In [11]:
print(result.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
movie_input (InputLayer)        (None, 1)            0                                            
__________________________________________________________________________________________________
user_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
movie_feature_space (Embedding) (None, 1, 30)        50460       movie_input[0][0]                
__________________________________________________________________________________________________
User_Feature_space (Embedding)  (None, 1, 30)        28290       user_input[0][0]                 
__________________________________________________________________________________________________
flatten_7 

In [12]:
#We will make 10 iterations through full train dataset
result.fit([train.user_id, train.item_id], train.rating, epochs=10)


Epoch 1/10
80000/80000 [==============================] - 19s 239us/step - loss: 1.6480
Epoch 2/10
80000/80000 [==============================] - 19s 240us/step - loss: 1.1084
Epoch 3/10
80000/80000 [==============================] - 17s 216us/step - loss: 1.0049
Epoch 4/10
80000/80000 [==============================] - 17s 209us/step - loss: 0.9470
Epoch 5/10
80000/80000 [==============================] - 17s 217us/step - loss: 0.9092
Epoch 6/10
80000/80000 [==============================] - 17s 215us/step - loss: 0.8824
Epoch 7/10
80000/80000 [==============================] - 17s 215us/step - loss: 0.8649
Epoch 8/10
80000/80000 [==============================] - 17s 216us/step - loss: 0.8502
Epoch 9/10
80000/80000 [==============================] - 17s 217us/step - loss: 0.8371
Epoch 10/10
80000/80000 [==============================] - 19s 243us/step - loss: 0.8234


In [13]:
predicted = result.predict([train.user_id, train.item_id])
actual = train.rating
print()
print('###The error in Training set is: '+str(mean_absolute_error(predicted, actual)))
predicted = result.predict([test.user_id, test.item_id])
actual = test.rating
print()
print('###The error in Test set is: '+str(mean_absolute_error(predicted, actual)))



###The error in Training set is: 0.6721769634380936

###The error in Test set is: 0.7284379516422749
